In [1]:
from selenium import webdriver
import time
import pandas as pd
import json

In [188]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()

In [189]:
driver.get('http://jv.acb.com/es/101266/cartadetiro')
driver.implicitly_wait(10)

Tener en cuenta que nada más entrar tengo que ir a **jornada** y ya luego empezar el loop

### 1.1 We are gonna find the name of the players and, then, collect the information in a dictionary for every shot from every player. 

In [201]:
data = get_shots()

### 1.3 Now we are gonna find how to go from a game to another

In [230]:
#Ya tenemos el shots.csv
def season(first_match, last_match):
    final = {}
    for i in range (1, 35):
        try:
            semana = driver.find_element_by_class_name("nav-top-sec-round").click()
            nueva = driver.find_elements_by_class_name("nav-opt")
            nueva[-(i)].click()
            datos_jornada = {}
            intermedio = []
            buscador = driver.find_element_by_class_name("nav-cont.nav-cont--day")
            jornada = buscador.find_elements_by_class_name("nav-day-match")
            for game in jornada:
                game.click()
                data = codigazo()
                datos_jornada.update(data)
            intermedio.append(datos_jornada)
            final[i] = intermedio
        except TypeError:
            raise "This game has not started yet"
            
    return final

In [225]:
data_items = final.items()
data_list = list(data_items)

df = pd.DataFrame(data_list)

In [226]:
df

,0,1
0,1,"[{'A. Della Valle': ['#local-out', '104.418306..."
1,2,"[{'J. Zyskowski': ['#local-in', '24.75', '126...."
2,3,"[{'D. DiLeo': ['#local-out', '166.684342478713..."
3,4,"[{'J. Rousselle': ['#local-in', '24.75', '134...."
4,5,"[{'J. Vaulet': ['#local-dunk', '24.75', '141.2..."
5,6,"[{'C. Hannah': ['#local-in', '93.8345789971617..."
6,7,"[{'S. Eatherton': ['#local-in', '26.6606906338..."
7,8,"[{'J. Feldeine': ['#local-in', '168.6137653736..."
8,9,"[{'J. Feldeine': ['#local-in', '170.0561494796..."
9,10,"[{'D. DiLeo': ['#local-out', '154.939214758751..."


In [227]:
df.to_csv("./data/shots_v1.csv")